 1. Import Required Libraries

In [ ]:
# TensorFlow and model building
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# For data handling and visualization
import matplotlib.pyplot as plt
import os


: 